# Welcome to the Day 2 Lab!


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get started --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

## First - let's talk about the Chat Completions API

1. The simplest way to call an LLM
2. It's called Chat Completions because it's saying: "here is a conversation, please predict what should come next"
3. The Chat Completions API was invented by OpenAI, but it's so popular that everybody uses it!

### We will start by calling OpenAI again - but don't worry non-OpenAI people, your time is coming!


In [1]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


## Do you know what an Endpoint is?

If not, please review the Technical Foundations guide in the guides folder

And, here is an endpoint that might interest you...

In [2]:
import requests

headers = {"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"}

payload = {
    "model": "gpt-5-nano",
    "messages": [
        {"role": "user", "content": "Tell me a fun fact"}]
}

payload

{'model': 'gpt-5-nano',
 'messages': [{'role': 'user', 'content': 'Tell me a fun fact'}]}

In [3]:
response = requests.post(
    "https://api.openai.com/v1/chat/completions",
    headers=headers,
    json=payload
)

response.json()

{'id': 'chatcmpl-DCZESys4QaIZRPKo3rq8i7IpzcD78',
 'object': 'chat.completion',
 'created': 1771887740,
 'model': 'gpt-5-nano-2025-08-07',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'Fun fact: On Venus, a day (one rotation) is longer than a year (one orbit around the Sun). A Venus day is about 243 Earth days, while its year is about 225 Earth days. Want another fun fact?',
    'refusal': None,
    'annotations': []},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 11,
  'completion_tokens': 890,
  'total_tokens': 901,
  'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0},
  'completion_tokens_details': {'reasoning_tokens': 832,
   'audio_tokens': 0,
   'accepted_prediction_tokens': 0,
   'rejected_prediction_tokens': 0}},
 'service_tier': 'default',
 'system_fingerprint': None}

In [4]:
response.json()["choices"][0]["message"]["content"]

'Fun fact: On Venus, a day (one rotation) is longer than a year (one orbit around the Sun). A Venus day is about 243 Earth days, while its year is about 225 Earth days. Want another fun fact?'

# What is the openai package?

It's known as a Python Client Library.

It's nothing more than a wrapper around making this exact call to the http endpoint.

It just allows you to work with nice Python code instead of messing around with janky json objects.

But that's it. It's open-source and lightweight. Some people think it contains OpenAI model code - it doesn't!


In [5]:
# Create OpenAI client

from openai import OpenAI
openai = OpenAI()

response = openai.chat.completions.create(model="gpt-5-nano", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content



'Fun fact: Honey never spoils. Archaeologists have found edible honey in 3,000-year-old jars from ancient Egypt. Its low water content, acidic pH, and natural enzymes help prevent spoilage. Want a fun fact on a specific topic? Space, animals, or history?'

## And then this great thing happened:

OpenAI's Chat Completions API was so popular, that the other model providers created endpoints that are identical.

They are known as the "OpenAI Compatible Endpoints".

For example, google made one here: https://generativelanguage.googleapis.com/v1beta/openai/

And OpenAI decided to be kind: they said, hey, you can just use the same client library that we made for GPT. We'll allow you to specify a different endpoint URL and a different key, to use another provider.

So you can use:

```python
gemini = OpenAI(base_url="https://generativelanguage.googleapis.com/v1beta/openai/", api_key="AIz....")
gemini.chat.completions.create(...)
```

And to be clear - even though OpenAI is in the code, we're only using this lightweight python client library to call the endpoint - there's no OpenAI model involved here.

If you're confused, please review Guide 9 in the Guides folder!

And now let's try it!

## THIS IS OPTIONAL - but if you wish to try out Google Gemini, please visit:

https://aistudio.google.com/

And set up your API key at

https://aistudio.google.com/api-keys

And then add your key to the `.env` file, being sure to Save the .env file after you change it:

`GOOGLE_API_KEY=AIz...`


In [6]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"

load_dotenv(override=True)

google_api_key = os.getenv("GOOGLE_API_KEY")

if not google_api_key:
    print("No API key was found - please be sure to add your key to the .env file, and save the file! Or you can skip the next 2 cells if you don't want to use Gemini")
elif not google_api_key.startswith("AIz"):
    print("An API key was found, but it doesn't start AIz")
else:
    print("API key found and looks good so far!")



No API key was found - please be sure to add your key to the .env file, and save the file! Or you can skip the next 2 cells if you don't want to use Gemini


In [ ]:
gemini = OpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)

response = gemini.chat.completions.create(model="gemini-2.5-flash-lite", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content

## And Ollama also gives an OpenAI compatible endpoint

...and it's on your local machine!

If the next cell doesn't print "Ollama is running" then please open a terminal and run `ollama serve`

In [7]:
requests.get("http://localhost:11434").content

b'Ollama is running'

### Download llama3.2 from meta

Change this to llama3.2:1b if your computer is smaller.

Don't use llama3.3 or llama4! They are too big for your computer..

In [8]:
!ollama pull llama3.2

]11;?\pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [9]:
OLLAMA_BASE_URL = "http://localhost:11434/v1"

ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')


In [11]:
# Get a fun fact

response = ollama.chat.completions.create(model="llama3.2", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content

"Here's one:\n\nDid you know that honey never spoils? Archaeologists have found pots of honey in ancient Egyptian tombs that are over 3,000 years old and still perfectly edible! The unique properties of honey, including its low water content and acidic pH, make it virtually impossible for bacteria to grow, which is why it remains a stable food source. Isn't that sweet?"

In [12]:
# Now let's try deepseek-r1:1.5b - this is DeepSeek "distilled" into Qwen from Alibaba Cloud

!ollama pull deepseek-r1:1.5b

]11;?\pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏  23 KB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 461 KB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 956 KB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 1.4 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 2.0 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 3.0 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 3.3 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 4.1 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% ▕                  

In [14]:
response = ollama.chat.completions.create(model="deepseek-r1:1.5b", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content

"Sure! Here's a fun fact: **Twin Prime Conjecture**. It states that there are infinitely many primes \\( p \\) such that both \\( p \\) and \\( p + 2 \\) are prime numbers. These pairs are called twin primes, like (3,5) or (17,19). While mathematicians haven't proven whether there are infinitely many twin primes yet, they are pretty much the most famous unsolved problem in number theory!"

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [16]:
# Define the Ollama URL for use with the OpenAI python client Library

OLLAMA_BASE_URL = "http://localhost:11434/v1"

ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')

# Step 1: Create your prompts

system_prompt = """
    You are a helpful assistant who is given an email and may be a subject line. 
    You need to suggest a subject line that is catchy and  more relevant to the email.
    The subject line should be no more than 20 words.
    You should respond in markdown. Do not wrap the markdown in a code block. 
"""
user_prompt = """
    Here is the email:
    {email}
    Salary conversations are often treated as a moment to perform. Many job seekers prepare scripts, practice counteroffers, or focus on “winning” the negotiation. In reality, effective salary discussions begin long before numbers are exchanged. They are shaped by preparation, context, and how clearly you communicate your value.

The first mistake many candidates make is treating compensation as a single number. Employers do not. Internally, compensation decisions balance role scope, market data, team equity, budget cycles, and perceived risk. Understanding this helps job seekers approach the conversation with realism instead of emotion.

Market awareness is foundational. This means knowing not only general salary ranges, but also how your experience level, location, industry, and role focus affect compensation. Titles alone are misleading. Two people with the same title can be paid very differently based on impact, complexity, and accountability. Strong candidates ground their expectations in evidence, not assumptions.

Timing also matters. Salary conversations are most effective once alignment is established. When an employer sees clear fit and reduced risk, flexibility increases. This is why early interviews should focus on demonstrating relevance, problem solving, and results rather than leading with compensation demands.

Confidence in negotiation does not come from aggressiveness. It comes from clarity. Be prepared to explain how your skills, experience, and outcomes support your range. Ask thoughtful questions about growth, performance expectations, and total compensation. These signals show maturity and partnership.

Finally, remember that negotiation is not a test of worth. It is a business discussion. Approached with preparation and professionalism, it becomes a conversation about alignment, not confrontation.

When job seekers understand how compensation decisions are made, they negotiate with confidence and credibility.


    Here is the subject line:
    {subject_line}
    Negotiating Salary for a new job
"""

# Step 2: Make the messages list

messages = [{"role": "system", "content": system_prompt},{"role": "user", "content": user_prompt}] # fill this in

# Step 3: Call Ollama
#response = ollama.chat.completions.create(model="llama3.2", messages=messages)
response = ollama.chat.completions.create(model="deepseek-r1:1.5b", messages=messages)

# Step 4: print the result
print(response.choices[0].message.content)

**Subject Line:**  
"Strategies for Winning Pay Discussions: Preparation on Preparation on Value"


In [20]:
# imports

import os
from dotenv import load_dotenv
from scraper import fetch_website_contents
from IPython.display import Markdown, display
from openai import OpenAI

ed = "https://edwarddonner.com"
cbc = "https://cbc.ca"
cnn = "https://cnn.com"
website = fetch_website_contents(cnn)

# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = """
You are a snarky assistant that analyzes the contents of a website,
and provides a short, snarky, humorous summary, ignoring text that might be navigation related.
Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.
"""

# Define our user prompt

user_prompt_prefix = """
Here are the contents of a website.
Provide a short summary of this website.
If it includes news or announcements, then summarize these too.

"""


messages = [{"role": "system", "content": system_prompt},{"role": "user", "content": user_prompt_prefix + website}]

# Step 3: Call Ollama
#response = ollama.chat.completions.create(model="llama3.2", messages=messages)
response = ollama.chat.completions.create(model="deepseek-r1:1.5b", messages=messages)

# Step 4: print the result
summary = response.choices[0].message.content
#print(summary)
display(Markdown(summary))

To provide recommendations and clarity on navigating through CNN's website, here is an organized summary:

1. **Feedback Section**: At the bottom of each article or section, you can leave a comment by clicking "AD-Free" or "C Ann." visitors are encouraged to share their thoughts and suggestions.

2. **Technical Issues**: If your browser is slow or blocked from loading content, ensure your internet connection is fast and try refreshing the page in another window or device.

3. **Visiting Pages**: Some pages (like video and audio content) may take a moment for loading before displaying at the top of the screen. Be patient with this process as it's standard on most websites.

4. **Text Errors and Spelling**: Look for any typos that might have been overlooked in the editing process, as these are easy mistakes to make.

5. **Feedback Features**: Hundreds of check marks indicate strong attention by readers; you can leave additional comments there.

This helps visitors enhance their experience on CNN's site while recognizing the usual process associated with updating online content.